<a href="https://colab.research.google.com/github/hideaki-kyutech/softcomp2025/blob/main/Week4_DampingComparison_Report_SampleCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 4 Report Template
## Damping Comparison in CartPole (Same seed, same steps)

### Goal
Generate two videos with:
- the **same seed**
- the **same number of steps** (`CLIP_STEPS`)
but different damping strength.

### What you will edit (ONLY)
- `CLIP_STEPS`
- `k_weak`, `k_strong`
- `MAX_SEED_SEARCH`

### Outputs
- `controller_W_weak_damping.mp4`
- `controller_S_strong_damping.mp4`


## Cell 1 — Install dependencies
We install `gymnasium` and `ffmpeg` (required to save MP4).

In [1]:
!pip -q install gymnasium
!apt-get -qq install -y ffmpeg

## Cell 2 — Import libraries
These imports are needed for simulation and MP4 creation.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import gymnasium as gym
from google.colab import files

## Cell 3 — Student edit area (ONLY)
Edit only the parameters below. Keep everything else unchanged so that the comparison is fair.

- `CLIP_STEPS`: length of both videos (same value for both)
- `k_weak`, `k_strong`: damping strength (this is what you compare)
- `MAX_SEED_SEARCH`: how long we search for a seed that works for both controllers


In [3]:
# ---------------------------
# STUDENT EDIT AREA (ONLY)
# ---------------------------
CLIP_STEPS = 120     # keep same for both videos
k_weak   = 0.6       # weak damping
k_strong = 1.6       # strong damping
MAX_SEED_SEARCH = 3000
# ---------------------------

## Cell 4 — Membership function (MF)
A membership function maps a numeric value to a degree in [0, 1].
We use a triangular MF for both θ and θ̇.


In [4]:
def triangular_mf(x, a, b, c):
    x = np.asarray(x, dtype=float)
    left = (x - a) / (b - a + 1e-12)
    right = (c - x) / (c - b + 1e-12)
    return np.maximum(np.minimum(left, right), 0.0)

## Cell 5 — MF parameters (fixed for fairness)
We keep MF parameters fixed so that the only difference is damping strength `k`.

- Angle θ: Negative / Zero / Positive (signed)
- Angular velocity θ̇: Negative / Zero / Positive (signed)


In [5]:
# Angle θ MFs (signed)
theta_neg  = (-0.5, -0.2, 0.0)
theta_zero = (-0.15, 0.0, 0.15)
theta_pos  = (0.0, 0.2, 0.5)

# Angular velocity θ_dot MFs (signed)
tdot_neg  = (-4.0, -1.5, 0.0)
tdot_zero = (-0.8, 0.0, 0.8)
tdot_pos  = (0.0, 1.5, 4.0)

## Cell 6 — Fuzzification (numeric → μ values)
Convert (θ, θ̇) into membership degrees for N/Z/P.


In [6]:
def fuzzify_signed(theta, theta_dot):
    mu_theta = {
        "N": float(triangular_mf(theta, *theta_neg)),
        "Z": float(triangular_mf(theta, *theta_zero)),
        "P": float(triangular_mf(theta, *theta_pos)),
    }
    mu_tdot = {
        "N": float(triangular_mf(theta_dot, *tdot_neg)),
        "Z": float(triangular_mf(theta_dot, *tdot_zero)),
        "P": float(triangular_mf(theta_dot, *tdot_pos)),
    }
    return mu_theta, mu_tdot

## Cell 7 — Controller (PD-like fuzzy score)
We convert μ values into a signed tendency:
- Positive tendency → push Right
- Negative tendency → push Left

Then we combine:
- angle tendency (position-like)
- velocity tendency (damping-like) multiplied by `k`


In [7]:
def signed_tendency(mu_dict):
    # Z is neutral; tendency is P - N
    return (mu_dict["P"] - mu_dict["N"])

def action_from_score(score):
    return 1 if score >= 0 else 0  # 1=Right, 0=Left

def controller(mu_theta, mu_tdot, k):
    score = signed_tendency(mu_theta) + k * signed_tendency(mu_tdot)
    return action_from_score(score)

## Cell 8 — Seed feasibility check
We need a seed that survives `CLIP_STEPS` for **both** k values.
This function checks whether the controller survives for the given seed.


In [8]:
def survives_n_steps(seed, n_steps, k):
    env = gym.make("CartPole-v1")
    obs, info = env.reset(seed=seed)
    env.action_space.seed(seed)

    for _ in range(n_steps):
        theta = float(obs[2])
        theta_dot = float(obs[3])
        mu_theta, mu_tdot = fuzzify_signed(theta, theta_dot)
        a = controller(mu_theta, mu_tdot, k)

        obs, reward, terminated, truncated, info = env.step(a)
        if terminated or truncated:
            env.close()
            return False

    env.close()
    return True

## Cell 9 — MP4 recording function
We run exactly `n_steps` steps and record frames using `rgb_array` rendering.
Then we export an MP4 using Matplotlib animation + ffmpeg.


In [9]:
def record_mp4(seed, n_steps, k, filename):
    env = gym.make("CartPole-v1", render_mode="rgb_array")
    obs, info = env.reset(seed=seed)
    env.action_space.seed(seed)

    frames = []
    for _ in range(n_steps):
        theta = float(obs[2])
        theta_dot = float(obs[3])
        mu_theta, mu_tdot = fuzzify_signed(theta, theta_dot)
        a = controller(mu_theta, mu_tdot, k)

        obs, reward, terminated, truncated, info = env.step(a)
        frames.append(env.render())

        if terminated or truncated:
            break

    env.close()

    fig, ax = plt.subplots()
    ax.axis("off")
    im = ax.imshow(frames[0])

    def init():
        im.set_data(frames[0])
        return (im,)

    def animate(i):
        im.set_data(frames[i])
        return (im,)

    anim = animation.FuncAnimation(
        fig, animate, init_func=init,
        frames=len(frames), interval=50, blit=True
    )
    anim.save(filename, fps=20)
    plt.close(fig)
    print("Saved:", filename, "frames:", len(frames))

## Cell 10 — Find a seed that works for BOTH weak & strong damping
If no seed is found:
- reduce `CLIP_STEPS` (e.g., 90), or
- increase `MAX_SEED_SEARCH`


In [10]:
good_seed = None
for seed in range(MAX_SEED_SEARCH):
    if survives_n_steps(seed, CLIP_STEPS, k_weak) and survives_n_steps(seed, CLIP_STEPS, k_strong):
        good_seed = seed
        break

print("Found seed:", good_seed)
assert good_seed is not None, "No seed found. Reduce CLIP_STEPS (e.g., 90) or increase MAX_SEED_SEARCH."

Found seed: 0


## Cell 11 — Record two videos (same seed, same steps)
We create:
- Weak damping video (k = k_weak)
- Strong damping video (k = k_strong)


In [11]:
record_mp4(good_seed, CLIP_STEPS, k_weak,   "controller_W_weak_damping.mp4")
record_mp4(good_seed, CLIP_STEPS, k_strong, "controller_S_strong_damping.mp4")

Saved: controller_W_weak_damping.mp4 frames: 120
Saved: controller_S_strong_damping.mp4 frames: 120


## Cell 12 — Download the MP4 files
Upload these two MP4 files to Moodle, and write your short answers (5–8 lines).


In [12]:
files.download("controller_W_weak_damping.mp4")
files.download("controller_S_strong_damping.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## ✅ Submission checklist
- [ ] Both MP4 files are generated
- [ ] Both videos use the same seed and the same CLIP_STEPS
- [ ] You wrote short answers explaining damping and the observed trade-off
